In [159]:
import os
import pandas as pd
from decimal import Decimal
import numpy as np
from datetime import datetime, timedelta
from dotenv import load_dotenv
import yfinance as yf


import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objs as go
import pytz  # Make sure to import pytz for timezone handling
import seaborn as sns


import requests
import csv
import json

import warnings

In [160]:
# load_dotenv()

# local_inv_directry = os.getenv("local_inv_directry")

In [161]:
# # yahoo finance free version

# today_date = datetime.today()
# window_days = 365*10 # This is 10 years in days window

# # Define the list of stock symbols
# stocks_list = [
#     'SPY'
#     ,'CNI'
#     ,'PM'
#     # ,'COST'
#     # ,'WMT'

#     ]

# # Fetch the data
# data = yf.download(
#     stocks_list
#     ,start=today_date - timedelta(days=window_days)
#     ,end=today_date)



# # The data contains multi-level columns, we'll focus on 'Adj Close' for adjusted closing prices
# adj_close = data['Adj Close']

# # Perform standardization using StandardScaler
# scaler = StandardScaler()
# standardized_data = scaler.fit_transform(adj_close)

# # Convert the normalized data back to a DataFrame
# standardized_data = pd.DataFrame(standardized_data, index=adj_close.index, columns=adj_close.columns)


# # Plotting the adjusted closing prices of the stocks
# plt.figure(figsize=(14, 7))

# for stock in stocks_list:
#     plt.plot(standardized_data[stock], label=stock)

# plt.title('Stock Prices Over Time')
# plt.xlabel('Date')
# plt.ylabel('Adjusted Closing Price')
# plt.legend()
# plt.grid(True)
# plt.show()


In [162]:
load_dotenv()

API_KEY = os.getenv("alpha_vantage_api_key")

In [163]:
# Addtional setting session
# Set display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")
pd.options.mode.copy_on_write = True

In [164]:
# Parameters section

alpha_vantage_api_key = API_KEY # FREE TIER API rate limit is 25 requests per day
ticker_symbols = [
    'MO'
    ,'PM'
    ,'CNI'
    ,'VZ'
    ,'JNJ'
    ,'XOM'
    ,'WM'

 ]

window_days = 365*10
start_date = datetime.today()
end_date = start_date - timedelta(days=window_days)

stock_return_consolidate_df = pd.DataFrame()
stock_daily_ts_consolidate_df = pd.DataFrame()

In [165]:

for symbol in ticker_symbols:


    # Daily quote section
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={alpha_vantage_api_key}&outputsize=full&datatype=json'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'Time Series (Daily)':


            selected_cols = [
                '4. close'
            ]

            Daily_stock_df = pd.DataFrame(value).transpose()[selected_cols] # tranpose the dataframe and sub select selected cols

            # Rename columns
            Daily_stock_df.rename(
                columns={
                    '4. close': f'{symbol}'
                    }
                ,inplace=True
                )
            
            Daily_stock_df[f'{symbol}'] = Daily_stock_df[f'{symbol}'].astype(str).apply(lambda x: float(x))
            Daily_stock_df[f'{symbol}'] = Daily_stock_df[f'{symbol}'].round(2)


    # Dividend section
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=DIVIDENDS&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'data':
            stock_dividend_df = pd.DataFrame(value)
            if len(value) > 0:
                stock_dividend_df = stock_dividend_df[['ex_dividend_date', 'amount']]

                # Rename columns
                stock_dividend_df.rename(
                        columns={
                            'ex_dividend_date': f'{symbol}_div_date'
                            ,'amount': f'{symbol}_div_amount'
                            }
                        ,inplace=True
                        )
                
                stock_dividend_df[f'{symbol}_div_amount'] = stock_dividend_df[f'{symbol}_div_amount'].astype(str).apply(lambda x: float(x))
                stock_dividend_df[f'{symbol}_div_amount'] = stock_dividend_df[f'{symbol}_div_amount'].round(2)
            else:
                stock_dividend_df[f'{symbol}_div_date'] = 0
                stock_dividend_df[f'{symbol}_div_amount'] = 0





    # Time window filter applied
    Daily_stock_df_filter_timewindow = Daily_stock_df[(pd.to_datetime(Daily_stock_df.index) >= end_date)
                                                & (pd.to_datetime(Daily_stock_df.index) <= start_date)
                                                ]
    
    stock_dividend_df_filter_timewindow = stock_dividend_df[(pd.to_datetime(stock_dividend_df[f'{symbol}_div_date']) >= end_date)
                                                    & (pd.to_datetime(stock_dividend_df[f'{symbol}_div_date']) <= start_date)
                                                    ]
    # Calculate the return %
    # return including dividend = (current_price - initial_price + total_dividend_during_period) / initial_price
    stock_return = round(
        (
            (
                Daily_stock_df_filter_timewindow[f'{symbol}'][0] 
                - Daily_stock_df_filter_timewindow[f'{symbol}'][-1] 
                + stock_dividend_df_filter_timewindow[f'{symbol}_div_amount'].sum()
            ) 
            / Daily_stock_df_filter_timewindow[f'{symbol}'][-1]
        )
    ,4
    )

    # assign value to columns in consolidate dataframe
    stock_return_consolidate_df.loc[0,f'{symbol}_{window_days/365}_yrs_return'] = stock_return
    stock_daily_ts_consolidate_df[f'{symbol}'] = Daily_stock_df_filter_timewindow[f'{symbol}']



stock_daily_ts_consolidate_normalized_df = stock_daily_ts_consolidate_df / stock_daily_ts_consolidate_df.iloc[-1]

In [166]:
stock_return_consolidate_df.transpose().sort_values(
    by=stock_return_consolidate_df.transpose().columns[0]
    ,ascending=False
    )

,0
WM_10.0_yrs_return,4.4589
CNI_10.0_yrs_return,0.9975
MO_10.0_yrs_return,0.9179
JNJ_10.0_yrs_return,0.8628
PM_10.0_yrs_return,0.8403
XOM_10.0_yrs_return,0.4186
VZ_10.0_yrs_return,0.2255


In [167]:
stock_daily_ts_consolidate_normalized_df = stock_daily_ts_consolidate_df / stock_daily_ts_consolidate_df.iloc[-1]


# Create a Plotly figure
fig = go.Figure()

# Add traces for each stock
for column in stock_daily_ts_consolidate_normalized_df.columns:
    fig.add_trace(go.Scatter(
        x=stock_daily_ts_consolidate_normalized_df.index,
        y=stock_daily_ts_consolidate_normalized_df[column],
        mode='lines',
        name=column
    ))

# Customize layout
fig.update_layout(
    title=f'Normalized Stock Prices in {window_days/365} years',
    xaxis_title='Date',
    yaxis_title='Normalized Price',
    legend_title='Stock',
    template='plotly_white'
)

# Show the plot
fig.show()

In [194]:
# beer sales volume analysis

url = f'https://www.ttb.gov/media/79096/download?inline'
r = requests.get(url)
data = r.json()

for key, value in data.items():
    # print(key)

    if key == 'Monthly Data':
        US_beer_df = pd.DataFrame(value)

        selected_cols = [
                        'CY_Month_Number'
                        ,'Year'
                        ,'Statistical_Group'
                        # ,'Count_IMs' # counts of industry members
                        ,'Value'
                        ]

        filter_1 = US_beer_df['Statistical_Group'].isin(['1-Production', '2-Removals', '4-Stocks On Hand End-of-Month'])
        filter_2 = US_beer_df['Statistical_Category'].isin(['0-Category Total'])


        US_beer_df_filtered = US_beer_df[selected_cols][
                                                        (filter_1) 
                                                        & (filter_2)
                                                        ]


        # Convert Year and Month into a single datetime column for better plotting
        US_beer_df_filtered['Year_Month'] = pd.to_datetime(US_beer_df_filtered['Year'].astype(str) + '-' + US_beer_df_filtered['CY_Month_Number'].astype(str) + '-01')



# Create the plot
fig = px.line(
    US_beer_df_filtered
    ,x='Year_Month'
    ,y='Value'
    ,color='Statistical_Group'
    ,title='Line Chart of US Beers by Statistical Group'
    ,labels={
        'Year_Month': 'Year-Month'
        ,'Value': 'Volume'
        }
                  )

fig.show()